In [ ]:
import json
import pandas as pd
import numpy as np
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf
simpler_sf.simple_salesforce()
from sqlalchemy import create_engine

In [ ]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [180]:
ruts = "F:/martin/MetLife SFTP/derty_Metlife.xlsx"
rutsMetlife2 = pd.read_excel(ruts)


In [181]:
rutsMetlife2.head()


,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,FNAC,SEXO,DIRECCION,COMUNA,CIUDAD,TELEFONO,RELACION,DESDE,HASTA,NOMBRE,EMAIL,NROCELULAR,VIGENCIA
0,340007997,3391629,9,0,0,13707.0,F,EDUARDO ALERT 6229,LA REINA,SANTIAGO,965770703,CO,42064.0,NaN,Beltrán Martínez Ana Eliana,luarayabeltran@gmail.com,965770703,45351
1,340007997,4888346,K,0,0,18488.0,F,BOCACCIO 544,LAS CONDES,SANTIAGO,2204262- 098740509,CO,42064.0,NaN,Cabrera Undurraga Maria,NaN,NaN,45351
2,340007997,5393043,3,0,0,17977.0,F,Los Fresnos 1355 Casa 2 Condominio La Llanura,HUECHURABA,SANTIAGO,56-2-22488062,CO,42064.0,NaN,Muñoz Garate Edith Margot,AMADORITURRIETA@GMAIL.COM,56992279688,45351
3,340007997,5473602,9,0,0,19138.0,F,AVENIDA FRANCISCO DE VILLAGRA 5824,LA REINA,SANTIAGO,NaN,CO,42064.0,NaN,Contreras Mendoza Patricia,NaN,NaN,45351
4,340006458,6410825,5,0,0,21158.0,F,MAIPU 470 DP 504,CONCEPCIÓN,NaN,235951,CO,41974.0,NaN,Delgado Muñoz Olivia,docampo@udec.cl,NaN,45535


In [182]:
rutsMetlife = rutsMetlife2[
    [
        "POLIZA",
        "RUTASEGURADO",
        "DV",
        "RUTBENEFICIARIO",
        "RUTBENEFIARIO_DV",
        "RELACION",
    ]
]
rutsMetlife.shape

(744401, 6)

In [183]:
def siPrimerCero(y):
    if len(y) == 2:
        if y[0] == "0":
            y = y[1:]
    return y

In [184]:
rutsMetlife.head()

,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,RELACION
0,340007997,3391629,9,0,0,CO
1,340007997,4888346,K,0,0,CO
2,340007997,5393043,3,0,0,CO
3,340007997,5473602,9,0,0,CO
4,340006458,6410825,5,0,0,CO


In [185]:

rutsMetlife["RUTBENEFIARIO_DV"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).astype(str).apply(lambda y: siPrimerCero(y))
rutsMetlife.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\3045764418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutsMetlife["RUTBENEFIARIO_DV"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).astype(str).apply(lambda y: siPrimerCero(y))


,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,RELACION
0,340007997,3391629,9,0,0,CO
1,340007997,4888346,K,0,0,CO
2,340007997,5393043,3,0,0,CO
3,340007997,5473602,9,0,0,CO
4,340006458,6410825,5,0,0,CO


In [186]:

rutsMetlife["Relación"] = rutsMetlife["RELACION"].astype(str)
rutsMetlife["Relación"] = np.where(rutsMetlife["Relación"]== "AS","Titular", "Carga")
rutsMetlife = rutsMetlife[rutsMetlife["Relación"]=="Titular" ]

rutsMetlife["Poliza"] = rutsMetlife["POLIZA"].astype(int).astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("-", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("–", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(".", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(" ", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.upper()
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.strip()

rutsMetlife["dv_asegurado"] = rutsMetlife["DV"].astype(str).apply(lambda y: siPrimerCero(y)) 
#rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace("-", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace("–", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].astype(str)
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace(".", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace(" ", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.upper()
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.strip()

rutsMetlife["Rut_Titular"] = rutsMetlife["RUTASEGURADO"].astype(str).apply(lambda y: siPrimerCero(y)) 
#rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str).apply(lambda y: siPrimerCero(y)) 
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("-", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("–", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(".", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(" ", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.upper()
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.strip()

rutsMetlife["dv_Beneficiario"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).apply(lambda y: siPrimerCero(y))
#rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace("-", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace("–", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].astype(str)
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace(".", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace(" ", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.upper()
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["RUTBENEFICIARIO"].astype(int).astype(str).apply(lambda y: siPrimerCero(y))
#rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("-", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("–", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(".", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(" ", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.upper()
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"] + rutsMetlife["dv_Beneficiario"]
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"] + rutsMetlife["dv_asegurado"]


#Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]


#rutsMetlife["llave"] = rutsMetlife["Poliza"] + rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]
rutsMetlife["llave"] = rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]

""" rutsMetlife = rutsMetlife[
    ["Poliza",	"Rut_Titular",	"Rut_Beneficiario",	"Relación","llave",
       
    ]
] """
rutsMetlife = rutsMetlife.drop_duplicates(
    [
        "llave",	
    ],
    keep="last",
)
rutsMetlife.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\683568310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutsMetlife["Relación"] = rutsMetlife["RELACION"].astype(str)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\683568310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutsMetlife["Relación"] = np.where(rutsMetlife["Relación"]== "AS","Titular", "Carga")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\683568310.py:9: FutureWarning: The default value of regex will change from True t

(303017, 13)

In [187]:
rutsMetlife.head()

,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,RELACION,Relación,Poliza,dv_asegurado,Rut_Titular,dv_Beneficiario,Rut_Beneficiario,llave
2103,340006458,12345,5,12345,5,AS,Titular,340006458,5,123455,5,123455,123455123455
2116,340020148,1039122,9,1039122,9,AS,Titular,340020148,9,10391229,9,10391229,1039122910391229
2121,340024778,1446411,5,1446411,5,AS,Titular,340024778,5,14464115,5,14464115,1446411514464115
2125,340006130,1543124,5,1543124,5,AS,Titular,340006130,5,15431245,5,15431245,1543124515431245
2128,340011238,1701221,5,1701221,5,AS,Titular,340011238,5,17012215,5,17012215,1701221517012215


In [188]:

#ruts_metlife = rutsMetlife
rutsMetlife = rutsMetlife.sort_values(by="llave", ascending = True)
#rutsMetlife.to_excel("F:/martin/MetLife SFTP/Titulares unicos Metlife.xlsx", index=False)

In [189]:
query_empresacontactoconvenio = (
    "select N_Documento_Titular__c,Paciente__r.IdentificationId__pc from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = 'MetLife' and Relaci_n_Con_Asegurado__c = 'Titular' and Convenio_Activo__c = true "
    
)


#and Convenio_Activo__c = true 
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
#empresacontactoconvenio.

#empresacontactoconvenio = rutsMetlife

100%|██████████| 1/1 [02:01<00:00, 121.13s/it]


In [ ]:
""" ruts = "F:/martin/MetLife SFTP/derty_metlife_diciembre.xlsx"
rutsMetlife = pd.read_excel(ruts)

rutsMetlife["Relación"] = rutsMetlife["RELACION"].astype(str)
rutsMetlife["Relación"] = np.where(rutsMetlife["Relación"]== "AS","Titular", "Carga")
rutsMetlife = rutsMetlife[rutsMetlife["Relación"]=="Titular" ]

rutsMetlife["Poliza"] = rutsMetlife["POLIZA"].astype(int).astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("-", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("–", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(".", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(" ", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.upper()
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.strip()

rutsMetlife["dv_asegurado"] = rutsMetlife["DV"].astype(str).apply(lambda y: siPrimerCero(y)) 
#rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace("-", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace("–", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].astype(str)
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace(".", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace(" ", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.upper()
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.strip()

rutsMetlife["Rut_Titular"] = rutsMetlife["RUTASEGURADO"].astype(str).apply(lambda y: siPrimerCero(y)) 
#rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str).apply(lambda y: siPrimerCero(y)) 
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("-", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("–", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(".", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(" ", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.upper()
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.strip()

rutsMetlife["dv_Beneficiario"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).apply(lambda y: siPrimerCero(y))
#rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace("-", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace("–", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].astype(str)
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace(".", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace(" ", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.upper()
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["RUTBENEFICIARIO"].astype(int).astype(str).apply(lambda y: siPrimerCero(y))
#rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("-", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("–", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(".", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(" ", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.upper()
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"] + rutsMetlife["dv_Beneficiario"]
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"] + rutsMetlife["dv_asegurado"]


#Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]


#rutsMetlife["llave"] = rutsMetlife["Poliza"] + rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]
rutsMetlife["llave"] = rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]

rutsMetlife = rutsMetlife[
    ["Poliza",	"Rut_Titular",	"Rut_Beneficiario",	"Relación","llave",
       
    ]
] 
rutsMetlife = rutsMetlife.drop_duplicates(
    [
        "llave",	
    ],
    keep="last",
)
rutsMetlife.shape

 """

In [ ]:
""" empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].astype(str)
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace("-", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace("–", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].astype(str)
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace(".", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace(" ", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.upper()
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.strip()

empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].astype(str)
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace("-", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace("–", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].astype(str)
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace(".", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace(" ", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.upper()
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.strip()

empresacontactoconvenio.head() """

In [190]:
empresacontactoconvenio["llave"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"] + empresacontactoconvenio["N_Documento_Titular__c"]
empresacontactoconvenio = empresacontactoconvenio.drop_duplicates(
    [
        "llave",	
    ],
    keep="last",
)
empresacontactoconvenio.shape

(338576, 3)

In [191]:
empresacontactoconvenio["Esta"] = empresacontactoconvenio["llave"].isin(rutsMetlife["llave"])
empresacontactoconvenio = empresacontactoconvenio[empresacontactoconvenio["Esta"]==False]
#rutsMetlife = rutsMetlife[rutsMetlife["Relación"]=="Titular" ]
empresacontactoconvenio.shape

(36365, 4)

In [192]:

empresacontactoconvenio.to_excel("F:/martin/MetLife SFTP/diferencia_36000.xlsx", index=False) 

In [ ]:
""" ruts_metlife = pd.concat([ruts_metlife, rutsMetlife], axis=0)
ruts_metlife = ruts_metlife.drop_duplicates(
    [
        "Poliza","Rut_Titular",	"Rut_Beneficiario",	"Relación",
    ],
    keep="last",
)
ruts_metlife.shape """

In [ ]:

""" reportes_metlife = "C:/Users/jmartinez/Documents/Metlife_derty.xlsx"
reportes_metlife_fredy = pd.read_excel(reportes_metlife) """

In [ ]:
""" reportes_metlife_fredy["llave"] =  reportes_metlife_fredy["Rut"]
ruts_metlife["LLave"] = ruts_metlife["Rut_Beneficiario"]  """

In [ ]:
""" reportes_metlife_fredy = pd.merge(
    left=reportes_metlife_fredy,
    right=ruts_metlife,
    how="left",
    left_on="llave",
    right_on="LLave",
) """

In [ ]:
""" reportes_metlife_fredy.head(5) """

In [ ]:
#reportes_metlife_fredy.to_excel("C:/Users/jmartinez/Documents/reportes_metlife2.xlsx", index=False)